ETP, exchange traded products, track underlying securities, indexes, or other financial instruments, similarly to stock these are traded on exchanges which implies daily price fluctuations.Each ETP can be include as many underelying assets, digital ones also, as the creator decides, thus, gaining and losing in value in sync with the underlying assets, ETFs are the msot popular types of ETPs. These financial basket type of instruments are a great cost-effective way for institutions and individuals to gain exposure to a specific market, therefore crypto ETPs are great for investors willing to gain an exposure to crypto without the hassle of holding coins on spot or wallets.

The following is an EDA, exploratory data analysis, aiming for insights about:

ETP's liquidity (bid/ask spreads & volume)
Underlying assets' liquidity (bid/ask spreads & volume)
Relevant correlations

In [70]:
import pandas as pd
import pandas_datareader as web
import numpy as np
from datetime import datetime
import seaborn as sns

from matplotlib import pyplot as plt
import matplotlib as matplot
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import requests
import json
from pandas.plotting import register_matplotlib_converters

#ETP csv file downloaded from six-group website to extract a list of ETP names & ISIN (Internation Security Identification Numbers)
df = pd.read_csv('ETP Explorer export_splittedintocolumns.csv')
# df.info()
df

,FundLongName,ValorSymbol,FundReutersTicker,FundBloombergTicker,ValorNumber,ISIN,IssuerLongNameDesc,IssuerNameFull,TradingBaseCurrency,FundCurrency,...,DailyLowPrice,OnMarketVolume,OffBookVolume,TotalTurnover,TotalTurnoverCHF,ProductLineDesc,AssetClassDesc,UnderlyingGeographicalDesc,LegalStructureCountryDesc,UnderlyingProviderDesc
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,BTCB.S,BTCB SE,113027698,CH1130276988,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,ETHB.S,ETHB SE,113027699,CH1130276996,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,EUR,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,USD,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,21Shares Algorand (ALGO) ETP,ALGO,ALGO.S,ALGO SE,114688231,CH1146882316,21Shares AG,21Shares AG,CHF,USD,...,8.8760,484.0,0.0,4371.18,4371.18,Exchange Traded Product,Other,Global,Switzerland,NaN
6,21Shares Algorand (ALGO) ETP,ALGO,ALGO.S,ALGO SE,114688231,CH1146882316,21Shares AG,21Shares AG,EUR,USD,...,NaN,0.0,0.0,0.00,0.00,Exchange Traded Product,Other,Global,Switzerland,NaN
7,21Shares Algorand (ALGO) ETP,ALGO,ALGO.S,ALGO SE,114688231,CH1146882316,21Shares AG,21Shares AG,USD,USD,...,9.8660,3714.0,0.0,37039.37,34091.04,Exchange Traded Product,Other,Global,Switzerland,NaN
8,21Shares Avalanche (AVAX) ETP,AVAX,AVAX.S,AVAX.SE,113520208,CH1135202088,21Shares AG,21Shares AG,CHF,USD,...,15.8600,1425.0,0.0,23213.75,23213.75,Exchange Traded Product,Other,Global,Switzerland,NaN
9,21Shares Avalanche (AVAX) ETP,AVAX,AVAX.S,AVAX.SE,113520208,CH1135202088,21Shares AG,21Shares AG,EUR,USD,...,14.9280,1157.0,0.0,18055.70,18900.71,Exchange Traded Product,Other,Global,Switzerland,NaN


In order to have a list of inputs that is recognizable by Yahoo Finance API, we need to add a .SW to each line in the VALORSYMBOL column.

In [71]:
df['.SW'] = '.SW'
df['YahooTicker'] = df['ValorSymbol'] + df['.SW']

Checking the columns' names to know what to drop and keep.

In [72]:
#Let's look at a preview of our data
print(df.columns)

Index(['FundLongName', 'ValorSymbol', 'FundReutersTicker',
       'FundBloombergTicker', 'ValorNumber', 'ISIN', 'IssuerLongNameDesc',
       'IssuerNameFull', 'TradingBaseCurrency', 'FundCurrency',
       'ManagementFee', 'ReplicationMethodDesc', 'ManagementStyleDesc',
       'MarketMakers', 'ClosingPrice', 'ClosingPerformance', 'ClosingDelta',
       'FundUnderlyingDescription', 'BidVolume', 'BidPrice', 'AskVolume',
       'AskPrice', 'MidSpread', 'PreviousClosingPrice', 'MarketDate',
       'MarketTime', 'OpeningPrice', 'OpeningPrice.1', 'DailyLowPrice',
       'OnMarketVolume', 'OffBookVolume', 'TotalTurnover', 'TotalTurnoverCHF',
       'ProductLineDesc', 'AssetClassDesc', 'UnderlyingGeographicalDesc',
       'LegalStructureCountryDesc', 'UnderlyingProviderDesc', '.SW',
       'YahooTicker'],
      dtype='object')


If we fetch the data from yahooFinance, we care about the columns 'YahooTicker', 'FundLongName', 'ISIN', 'FundCurrency's,'FundUnderlyingDecription'

In [73]:
#Dropping unnecessary columns

df.drop(columns = ['FundReutersTicker',
       'FundBloombergTicker', 'ValorNumber', 'IssuerLongNameDesc',
       'IssuerNameFull', 'TradingBaseCurrency', 
       'ManagementFee', 'ReplicationMethodDesc', 'ManagementStyleDesc',
       'MarketMakers', 'ClosingPrice', 'ClosingPerformance', 'ClosingDelta',
       'BidVolume', 'BidPrice', 'AskVolume',
       'AskPrice', 'MidSpread', 'PreviousClosingPrice', 'MarketDate',
       'MarketTime', 'OpeningPrice', 'OpeningPrice.1', 'DailyLowPrice',
       'OnMarketVolume', 'OffBookVolume', 'TotalTurnover', 'TotalTurnoverCHF',
       'ProductLineDesc', 'AssetClassDesc', 'UnderlyingGeographicalDesc',
       'LegalStructureCountryDesc', 'UnderlyingProviderDesc'], axis = 1, inplace = True)

#Option to view all rows to check for errors in YahooTicker column
pd.set_option('display.max_rows', None)
df

,FundLongName,ValorSymbol,ISIN,FundCurrency,FundUnderlyingDescription,.SW,YahooTicker
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,CH1130276988,CHF,Bitcoin (BTC),.SW,BTCB.SW
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,CH1130276996,CHF,Ethereum (ETH),.SW,ETHB.SW
2,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
3,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
4,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
5,21Shares Algorand (ALGO) ETP,ALGO,CH1146882316,USD,NaN,.SW,ALGO.SW
6,21Shares Algorand (ALGO) ETP,ALGO,CH1146882316,USD,NaN,.SW,ALGO.SW
7,21Shares Algorand (ALGO) ETP,ALGO,CH1146882316,USD,NaN,.SW,ALGO.SW
8,21Shares Avalanche (AVAX) ETP,AVAX,CH1135202088,USD,NaN,.SW,AVAX.SW
9,21Shares Avalanche (AVAX) ETP,AVAX,CH1135202088,USD,NaN,.SW,AVAX.SW


Next we will import the list of ETP that give us errors, using this list we will remove the rows containing the faulty tickers from our initial DF in order to fetch data from yahoo finance with issues.

In [74]:
errors_tickers_list = pd.read_csv('errors_list.csv')
errors_tickers_list.drop(['Unnamed: 0'], axis=1, inplace=True)
errors_tickers = errors_tickers_list['0'].tolist()
print(errors_tickers)

# for t in df['ValorSymbol']:
#     if t == errors_tickers[]:


df = df[~df.ValorSymbol.isin(errors_tickers)]
print(df)

['TPLT', 'WALT', 'TNIK', 'AVAX', 'TSLV', 'PLKA', 'BTIC', 'VAVA', 'LUNA', 'LITE', 'VBTC', 'RDAN', 'ALTS', 'TCOP', 'VETH', 'ADOT', 'ESOL', 'BTCE', 'VTOP', 'POLY', 'VDOT', 'SGLE', 'ELTC', 'VSOL', 'BLOC', 'TPAL', 'ATOM', 'TGLD', 'XBTI', 'ZETH', 'MEGA', 'ALGO']
                                          FundLongName ValorSymbol  \
0       1 FiCAS Active Bitcoin Exchange Traded Product        BTCB   
1      1 FiCAS Active Ethereum Exchange Traded Product        ETHB   
2                           15 FiCAS Active Crypto ETP        BTCA   
3                           15 FiCAS Active Crypto ETP        BTCA   
4                           15 FiCAS Active Crypto ETP        BTCA   
11                            21Shares Binance BNB ETP        ABNB   
12                           21Shares Bitcoin Cash ETP        ABCH   
13                                21Shares Bitcoin ETP        ABTC   
14                                21Shares Bitcoin ETP        ABTC   
15                                21Shares 

In [75]:

df[df["ValorSymbol"].str.contains("errors_tickers_list")==False]

# df = df[~df['ValorSymbol'].isin(['errors_tickers_list'])]
df

,FundLongName,ValorSymbol,ISIN,FundCurrency,FundUnderlyingDescription,.SW,YahooTicker
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,CH1130276988,CHF,Bitcoin (BTC),.SW,BTCB.SW
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,CH1130276996,CHF,Ethereum (ETH),.SW,ETHB.SW
2,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
3,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
4,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
11,21Shares Binance BNB ETP,ABNB,CH0496454155,USD,N/A,.SW,ABNB.SW
12,21Shares Bitcoin Cash ETP,ABCH,CH0475552201,USD,Bitcoin Cash,.SW,ABCH.SW
13,21Shares Bitcoin ETP,ABTC,CH0454664001,USD,Bitcoin,.SW,ABTC.SW
14,21Shares Bitcoin ETP,ABTC,CH0454664001,USD,Bitcoin,.SW,ABTC.SW
15,21Shares Bitcoin ETP,ABTC,CH0454664001,USD,Bitcoin,.SW,ABTC.SW


We will rely on a dictionary with YahooTicker as key values for data fetching from Yahoo Finance.
And we will also collect BTC data to add to our ETP database for analysis purpose.

Dictionary Creation, YahooTicker in keys & FundLongName in values.

In [76]:
#Creating our dicrtionary then assigning FundLongName and YahooTicker to a dictionary
#First assigning each column to a list 

ETP_dict = dict(zip(df.YahooTicker, df.FundLongName))

#Uncomment below line to preview of our dictionary
print(ETP_dict)

{'BTCB.SW': '1 FiCAS Active Bitcoin Exchange Traded Product', 'ETHB.SW': '1 FiCAS Active Ethereum Exchange Traded Product', 'BTCA.SW': '15 FiCAS Active Crypto ETP', 'ABNB.SW': '21Shares Binance BNB ETP', 'ABCH.SW': '21Shares Bitcoin Cash ETP', 'ABTC.SW': '21Shares Bitcoin ETP', 'ABBA.SW': '21Shares Bitcoin Suisse Index ETP', 'KEYS.SW': '21Shares Bitwise Select 10 Large Cap Crypto Index ETP', 'AADA.SW': '21Shares Cardano ETP', 'HODLX.SW': '21Shares Crypto Basket 10 ETP', 'HODLV.SW': '21Shares Crypto Basket Equal Weight ETP', 'HODL.SW': '21Shares Crypto Basket Index', 'AETH.SW': '21Shares Ethereum ETP', 'AXRP.SW': '21Shares Ripple XRP ETP', 'SBTC.SW': '21Shares Short Bitcoin ETP', 'ASOL.SW': '21Shares Solana (SOL) ETP', 'AXLM.SW': '21Shares Stellar ETP', 'AXTZ.SW': '21Shares Tezos ETP', 'BITC.SW': 'COINSHARES PHYSICAL BITCOIN (BTC)', 'ETHE.SW': 'COINSHARES PHYSICAL ETHEREUM (ETH)', 'XRPL.SW': 'COINSHARES PHYSICAL XRP', 'SGLD.SW': 'Invesco Physical Gold ETC', 'SBTCC.SW': 'SEBA Bitcoin ETP

In [77]:
#Collecting BTC daily data from yahoo finance
import yfinance as yf

#Setting today's date
today_date = datetime.today().strftime("%Y-%m-%d")
btc_df = yf.download("BTC-USD", start = "2021-01-01", end = today_date)
btc_df.drop(columns= ['Adj Close'], axis = 1, inplace = True)
btc_df.rename(columns= {"Open":"BTC-Open", "Close":"BTC-Close", "Volume" : "BTC-Volume"}, inplace = True)

delta = btc_df['BTC-Open'] - btc_df['BTC-Close']
btc_df['Delta'] = delta 
btc_df.head()

[*********************100%***********************]  1 of 1 completed


,BTC-Open,High,Low,BTC-Close,BTC-Volume,Delta
Date,,,,,,
2021-01-01,28994.009766,29600.626953,28803.585938,29374.152344,40730301359,-380.142578
2021-01-02,29376.455078,33155.117188,29091.181641,32127.267578,67865420765,-2750.812500
2021-01-03,32129.408203,34608.558594,32052.316406,32782.023438,78665235202,-652.615234
2021-01-04,32810.949219,33440.218750,28722.755859,31971.914062,81163475344,839.035156
2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,67547324782,-2015.388672


In [78]:
#Given the random nature of variations between OPEN & CLOSE we expect some negative Delta Hr Trend values 
#Verification of negative values existences
(btc_df < 0).any().any()

True

ETP_df is our final dataframe, including BTC data + ETPs data.

In [79]:
# pip install yfinance --upgrade --no-cache-dir

In [96]:
import fix_yahoo_finance

ETP_df = btc_df.copy()

ETP_dict
for key, value in ETP_dict.items():
    print(key, value)
    try:
        df_temp = yf.download(
            key, start = "2020-01-01", end = "2022-02-18", data_source = "yahoo"
        )
    except ValueError:
        print(f'{key} symbol not found')
    df_temp.index = pd.to_datetime(df_temp.index)
    df_temp.rename(columns = {"Close": value}, inplace = True)
    ETP_df = pd.merge(
        left = ETP_df,
        right = df_temp[value],
        how = 'inner',
        left_index = True, right_index = True)
ETP_df.head()



BTCB.SW 1 FiCAS Active Bitcoin Exchange Traded Product
[*********************100%***********************]  1 of 1 completed
ETHB.SW 1 FiCAS Active Ethereum Exchange Traded Product
[*********************100%***********************]  1 of 1 completed
BTCA.SW 15 FiCAS Active Crypto ETP
[*********************100%***********************]  1 of 1 completed
ABNB.SW 21Shares Binance BNB ETP
[*********************100%***********************]  1 of 1 completed
ABCH.SW 21Shares Bitcoin Cash ETP
[*********************100%***********************]  1 of 1 completed
ABTC.SW 21Shares Bitcoin ETP
[*********************100%***********************]  1 of 1 completed
ABBA.SW 21Shares Bitcoin Suisse Index ETP
[*********************100%***********************]  1 of 1 completed
KEYS.SW 21Shares Bitwise Select 10 Large Cap Crypto Index ETP
[*********************100%***********************]  1 of 1 completed
AADA.SW 21Shares Cardano ETP
[*********************100%***********************]  1 of 1 completed
HODL

,BTC-Open,High,Low,BTC-Close,BTC-Volume,Delta,1 FiCAS Active Bitcoin Exchange Traded Product,1 FiCAS Active Ethereum Exchange Traded Product,15 FiCAS Active Crypto ETP,21Shares Binance BNB ETP,...,SEBA Crypto Asset Select Index ETP (USD),SEBA Ethereum ETP (USD),SEBA Polkadot ETP (USD),Sygnum Platform Winners Index ETP,VanEck Polygon ETN,WisdomTree Bitcoin,WisdomTree Ethereum,Xtrackers Physical Gold ETC,Xtrackers Physical Gold EUR Hedged ETC,Xtrackers Physical Silver EUR Hedged ETC
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-16,48900.464844,49425.574219,47529.878906,47665.425781,27268150947,1235.039062,8.360,9.134,230.949997,40.000000,...,45.794998,4.0570,27.290001,24.090000,8.970,10.548,40.680000,174.059998,115.680000,134.600006
2021-12-17,47653.730469,48004.894531,45618.214844,46202.144531,32902725329,1451.585938,8.195,8.875,225.949997,39.345001,...,43.369999,3.8875,25.440001,23.059999,8.802,10.110,38.080002,174.679993,116.040001,136.539993
2021-12-20,46707.062500,47401.718750,45579.808594,46880.277344,30961902129,-173.214844,8.031,8.688,221.000000,38.049999,...,42.924999,3.8005,23.950001,22.450001,8.753,10.002,37.994999,174.320007,115.639999,135.080002
2021-12-21,46886.078125,49300.917969,46698.773438,48936.613281,27055803928,-2050.535156,8.485,8.903,228.750000,39.384998,...,45.000000,3.9400,24.639999,23.590000,9.543,10.592,39.435001,173.020004,114.980003,135.759995
2021-12-22,48937.097656,49544.796875,48450.941406,48628.511719,24447979559,308.585938,8.363,9.000,230.899994,39.709999,...,45.955002,3.9860,27.344999,24.040001,10.930,10.582,39.369999,174.020004,115.599998,137.259995


In [99]:
print(ETP_df.columns)

Index(['BTC-Open', 'High', 'Low', 'BTC-Close', 'BTC-Volume', 'Delta',
       '1 FiCAS Active Bitcoin Exchange Traded Product',
       '1 FiCAS Active Ethereum Exchange Traded Product',
       '15 FiCAS Active Crypto ETP', '21Shares Binance BNB ETP',
       '21Shares Bitcoin Cash ETP', '21Shares Bitcoin ETP',
       '21Shares Bitcoin Suisse Index ETP',
       '21Shares Bitwise Select 10 Large Cap Crypto Index ETP',
       '21Shares Cardano ETP', '21Shares Crypto Basket 10 ETP',
       '21Shares Crypto Basket Equal Weight ETP',
       '21Shares Crypto Basket Index', '21Shares Ethereum ETP',
       '21Shares Ripple XRP ETP', '21Shares Short Bitcoin ETP',
       '21Shares Solana (SOL) ETP', '21Shares Stellar ETP',
       '21Shares Tezos ETP', 'COINSHARES PHYSICAL BITCOIN (BTC)',
       'COINSHARES PHYSICAL ETHEREUM (ETH)', 'COINSHARES PHYSICAL XRP',
       'Invesco Physical Gold ETC', 'SEBA Bitcoin ETP (CHF - hedged)',
       'SEBA Bitcoin ETP (USD)', 'SEBA Crypto Asset Select Index ETP (U


'1 FiCAS Active Bitcoin Exchange Traded Product',
'1 FiCAS Active Ethereum Exchange Traded Product',
'15 FiCAS Active Crypto ETP', 
'21Shares Binance BNB ETP',
'21Shares Bitcoin Cash ETP', 
'21Shares Bitcoin ETP',
'21Shares Bitcoin Suisse Index ETP',
'21Shares Bitwise Select 10 Large Cap Crypto Index ETP',
'21Shares Cardano ETP', 
'21Shares Crypto Basket 10 ETP',
'21Shares Crypto Basket Equal Weight ETP',
'21Shares Crypto Basket Index', 
'21Shares Ethereum ETP',
'21Shares Ripple XRP ETP', 
'21Shares Short Bitcoin ETP',
'21Shares Solana (SOL) ETP', 
'21Shares Stellar ETP',
'21Shares Tezos ETP', 
'COINSHARES PHYSICAL BITCOIN (BTC)',
'COINSHARES PHYSICAL ETHEREUM (ETH)', 
'COINSHARES PHYSICAL XRP',
'Invesco Physical Gold ETC', 
'SEBA Bitcoin ETP (CHF - hedged)',
'SEBA Bitcoin ETP (USD)', 
'SEBA Crypto Asset Select Index ETP (USD)',
'SEBA Ethereum ETP (USD)', 
'SEBA Polkadot ETP (USD)',
'Sygnum Platform Winners Index ETP', 
'VanEck Polygon ETN',
'WisdomTree Bitcoin', 
'WisdomTree Ethereum',
'Xtrackers Physical Gold ETC', 
'Xtrackers Physical Gold EUR Hedged ETC',
'Xtrackers Physical Silver EUR Hedged ETC',

In [97]:
# YahooTicker_list = df['YahooTicker'].tolist()
# print(YahooTicker_list)

# data = yf.download(['BTCB.SW', 'ETHB.SW', 'BTCA.SW', 'BTCA.SW', 'BTCA.SW',], start = "2020-01-01", end = "2022-02-02", interval= '1d')
# data

# # print(data.head())

# """
# Getting info of each ticker in YahooTicker_list to check if there are exceptions.
# Exception mean we can not do much, we can not collect data about the ticker in question
# """
# # for t in YahooTicker_list:
# #     ticker = YahooTicker_list(t)
# #     info = None
    
# #     try:
# #         info = ticker.info
# #     except:
# #         print(f"Cannot get info of {t}, it probably has no data or does not exist on Yahoo Finance.")
# #         continue
# #     #Got the info of ticker 
# #     print(f"Info of {t}: {info}")


In [98]:
ETP_df.head()

,BTC-Open,High,Low,BTC-Close,BTC-Volume,Delta,1 FiCAS Active Bitcoin Exchange Traded Product,1 FiCAS Active Ethereum Exchange Traded Product,15 FiCAS Active Crypto ETP,21Shares Binance BNB ETP,...,SEBA Crypto Asset Select Index ETP (USD),SEBA Ethereum ETP (USD),SEBA Polkadot ETP (USD),Sygnum Platform Winners Index ETP,VanEck Polygon ETN,WisdomTree Bitcoin,WisdomTree Ethereum,Xtrackers Physical Gold ETC,Xtrackers Physical Gold EUR Hedged ETC,Xtrackers Physical Silver EUR Hedged ETC
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-16,48900.464844,49425.574219,47529.878906,47665.425781,27268150947,1235.039062,8.360,9.134,230.949997,40.000000,...,45.794998,4.0570,27.290001,24.090000,8.970,10.548,40.680000,174.059998,115.680000,134.600006
2021-12-17,47653.730469,48004.894531,45618.214844,46202.144531,32902725329,1451.585938,8.195,8.875,225.949997,39.345001,...,43.369999,3.8875,25.440001,23.059999,8.802,10.110,38.080002,174.679993,116.040001,136.539993
2021-12-20,46707.062500,47401.718750,45579.808594,46880.277344,30961902129,-173.214844,8.031,8.688,221.000000,38.049999,...,42.924999,3.8005,23.950001,22.450001,8.753,10.002,37.994999,174.320007,115.639999,135.080002
2021-12-21,46886.078125,49300.917969,46698.773438,48936.613281,27055803928,-2050.535156,8.485,8.903,228.750000,39.384998,...,45.000000,3.9400,24.639999,23.590000,9.543,10.592,39.435001,173.020004,114.980003,135.759995
2021-12-22,48937.097656,49544.796875,48450.941406,48628.511719,24447979559,308.585938,8.363,9.000,230.899994,39.709999,...,45.955002,3.9860,27.344999,24.040001,10.930,10.582,39.369999,174.020004,115.599998,137.259995
